In [1]:
import numpy as np
import pandas as pd

In [2]:
def spx_process(filename):
    raw = pd.read_csv(filename, 
                      names = ["datetime", "open", "high", "low", "close"])
    raw.datetime = pd.to_datetime(raw.datetime, format = '%Y.%m.%d %H:%M:%S')
    raw["date"] = raw.datetime.dt.date
    raw["time"] = raw.datetime.dt.time
    if filename == "SPX_2014_12.csv":
        data = raw.loc[raw.datetime > "2013-01-01",:]
    elif filename == "SPX_2016_12.csv":
        data = raw.loc[raw.datetime > "2015-01-01",:]
    else:
        data = raw
    before_930 = pd.to_timedelta(data.time.astype(str)) < pd.to_timedelta('09:30:00')
    after_359 = pd.to_timedelta(data.time.astype(str)) > pd.to_timedelta('15:59:00')
    delete_index = before_930 | after_359
    data = data.loc[~delete_index,:]
    data = data.reset_index().drop(columns = ["index", "time", "date"])
    return data

In [3]:
data1 = spx_process("../data/SPX_2014_12.csv")

In [4]:
data2 = spx_process("../data/SPX_2016_12.csv")

In [5]:
data3 = spx_process("../data/SPX_2018_12.csv")

In [6]:
data4 = spx_process("../data/SPX_2019_12.csv")

In [7]:
data = pd.concat([data1, data2, data3, data4], axis=0)

In [8]:
data.to_csv('../data/all_cleaned_SPX_data.csv', index=False)

In [9]:
data.head()

,datetime,open,high,low,close
0,2012-12-31 11:00:00,1405.22,1405.33,1405.14,1405.28
1,2012-12-31 11:01:00,1405.29,1406.35,1405.27,1406.35
2,2012-12-31 11:02:00,1406.59,1408.14,1406.59,1407.06
3,2012-12-31 11:03:00,1407.05,1407.89,1406.86,1407.89
4,2012-12-31 11:04:00,1407.77,1409.13,1407.77,1409.13
